In [2]:
import pandas as pd
import datetime
import numpy as np
import re
import sys

sys.path.append("..")
from aoc18.utils import read_input


# Day 13

In [88]:
inp = read_input(13, root_path="..")
inp_test = read_input(13, root_path="..", test=True)

In [78]:
straight = np.array([
    [1, 0],
    [0, 1]])

left = np.array([
    [ 0, 1],
    [-1, 0]])

right = np.array([
    [0, -1],
    [1,  0]])

anti_slash = np.array([
    [0, 1],
    [1, 0]])

slash = np.array([
   [ 0, -1],
   [-1,  0]])

def build_action(move_array):
    return np.vstack([
        np.hstack([np.identity(2), np.zeros((2,2))]),
        np.hstack([move_array]*2)
    ])

milestone_mapping = {
    "-": build_action(straight),
    "|": build_action(straight),
    "\\": build_action(anti_slash),
    "/": build_action(slash),
    ">": build_action(straight),
    "<": build_action(straight),
    "^": build_action(straight),
    "v": build_action(straight),
    "+": "intersection"
}

carts_mapping = {
    ">": np.array([ 0,  1]),
    "<": np.array([ 0, -1]),
    "v": np.array([ 1,  0]),
    "^": np.array([-1,  0])
}

intersections = [build_action(left), build_action(straight), build_action(right)]


class Milestone:
    
    def __init__(self, string):
        self.string = string
        self.action = self.get_action(string)
        
    def get_action(self, car):
        return milestone_mapping[car]
    
    def __str__(self):
        return self.string
    
    def __repr__(self):
        return "Milestone(%s)" % self.string

In [134]:
def get_collisions(carts_map):
    collisions = []
    for i in range(len(carts_map) - 1):
        for j in range(i + 1, len(carts_map)):
            if np.array_equal(carts_map[i]["cart"][:2], carts_map[j]["cart"][:2]):
                collisions.append((i,j))
    return collisions

def check_collision(carts_map):
    return len(get_collisions(carts_map))>0

def sort_carts(carts_map):
    return sorted(carts_map.items(), key=lambda kv: (kv[1]["cart"][0], kv[1]["cart"][1]))

In [135]:
def get_initial_state(inp):
    this_map = list(map(lambda i: list(map(lambda i: " ", range(len(inp[0][:-1])))), range(len(inp))))
    carts_map = {}
    cart_id = 0
    carts_order = []
    for x, line in enumerate(inp):
        for y, car in enumerate(line[:-1]):
            if car != " ":
                this_map[x][y] = Milestone(car)
            if car in carts_mapping:
                carts_map[cart_id] = {
                    "cart": np.hstack([np.array([x, y]), carts_mapping[car]]),
                    "status": 0
                }
                carts_order.append(cart_id)
                cart_id += 1
    return this_map, carts_map, carts_order

In [137]:
def run_one_step(this_map, carts_map, carts_order):
    for cart_id, cart in sort_carts(carts_map):
        milestone = this_map[cart["cart"][0]][cart["cart"][1]]
        if milestone.string == "+":
            action = intersections[np.mod(cart["status"], 3)]
            carts_map[cart_id]["status"] += 1
        else:
            action = milestone.action
        carts_map[cart_id]["cart"] = cart["cart"].dot(action).astype(int)
    return this_map, carts_map, carts_order

In [138]:
def run1(inp):
    this_map, carts_map, carts_order = get_initial_state(inp)
    iteration = 0
    while True:
        this_map, carts_map, carts_order = run_one_step(this_map, carts_map, carts_order)
        iteration += 1
        collisions = get_collisions(carts_map)
        if len(collisions) > 0:
            break
    cart_id_1, cart_id_2 = collisions[0]
    print("Collision after %d iterations! Cart %d and %d crashed at point (%d, %d)" % (
        iteration,
        cart_id_1,
        cart_id_2,
        carts_map[cart_id_1]["cart"][1],
        carts_map[cart_id_1]["cart"][0]
        )
    )
    return (carts_map[cart_id_1]["cart"][1], carts_map[cart_id_1]["cart"][0])
        


In [140]:
run1(inp)

Collision after 174 iterations! Cart 11 and 16 crashed at point (64, 86)


(64, 86)

In [141]:

carts_map

{0: {'cart': array([  3, 105,   0,  -1]), 'status': 0},
 1: {'cart': array([ 22, 105,   0,   1]), 'status': 0},
 2: {'cart': array([40, 90,  0, -1]), 'status': 0},
 3: {'cart': array([42, 27, -1,  0]), 'status': 0},
 4: {'cart': array([ 46, 130,   0,  -1]), 'status': 0},
 5: {'cart': array([67, 82,  0,  1]), 'status': 0},
 6: {'cart': array([67, 98,  0,  1]), 'status': 0},
 7: {'cart': array([71, 66,  1,  0]), 'status': 0},
 8: {'cart': array([ 92, 136,   1,   0]), 'status': 0},
 9: {'cart': array([ 94, 132,   1,   0]), 'status': 0},
 10: {'cart': array([99, 26,  1,  0]), 'status': 0},
 11: {'cart': array([100,  94,  -1,   0]), 'status': 0},
 12: {'cart': array([104,  67,   0,   1]), 'status': 0},
 13: {'cart': array([105,  81,  -1,   0]), 'status': 0},
 14: {'cart': array([105, 130,   0,  -1]), 'status': 0},
 15: {'cart': array([125, 103,   0,   1]), 'status': 0},
 16: {'cart': array([140, 124,   0,  -1]), 'status': 0}}

In [143]:
dict([("".join(map(str,v)),k) for k,v in carts_mapping.items()])

{'-10': '^', '0-1': '<', '01': '>', '10': 'v'}

In [144]:
inp[0]

'                           /-------------------------------------------------------------------------------------------------\\                        \n'

In [147]:
inp[86][64]

'+'

In [148]:
inp[85][64]

'|'

In [149]:
inp[86][65]

'-'

In [150]:
inp[8][136]

'|'